In [5]:
import os
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm import tqdm

def dhash(image, hash_size=8):
    """
    Tính toán hash của ảnh dựa trên chênh lệch gradient (Difference Hash).
    Cách này hiệu quả hơn Average Hash trong việc tìm ảnh tương đồng.
    """
    # 1. Resize và chuyển sang grayscale
    image = image.convert('L').resize((hash_size + 1, hash_size), Image.Resampling.LANCZOS)
    pixels = np.asarray(image)
    
    # 2. Tính chênh lệch giữa các pixel liền kề theo hàng ngang
    diff = pixels[:, 1:] > pixels[:, :-1]
    
    # 3. Chuyển boolean array sang hex string để làm mã định danh
    return "".join(["%1x" % sum([2**i for i, v in enumerate(row) if v]) for row in diff])

def find_duplicates(image_dir, threshold=2):
    """
    Tìm các ảnh trùng lặp hoặc gần giống nhau trong thư mục.
    threshold: số lượng bit khác biệt tối đa (0 là giống hệt).
    """
    image_dir = Path(image_dir)
    image_paths = list(image_dir.glob("*.[jJ][pP][gG]")) + \
                  list(image_dir.glob("*.[jJ][pP][eE][gG]")) + \
                  list(image_dir.glob("*.[pP][nN][gG]")) + \
                  list(image_dir.glob("*.[wW][eE][bB][pP]"))
    
    hashes = {}
    duplicates = []
    
    print(f"🔍 Đang quét {len(image_paths)} ảnh trong {image_dir}...")
    
    for path in tqdm(image_paths):
        try:
            with Image.open(path) as img:
                h = dhash(img)
                
                # So sánh với các hash đã lưu
                is_duplicate = False
                for existing_hash, existing_path in hashes.items():
                    # Tính khoảng cách Hamming (số lượng bit khác nhau)
                    distance = sum(c1 != c2 for c1, c2 in zip(h, existing_hash))
                    
                    if distance <= threshold:
                        duplicates.append((path, existing_path, distance))
                        is_duplicate = True
                        break
                
                if not is_duplicate:
                    hashes[h] = path
        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý {path}: {e}")
            
    return duplicates

def main():
    # Thay đổi đường dẫn đến thư mục ảnh của bạn ở đây
    target_dir = r"d:\Computer Vision\Computer-Vision Project\Computer-Vision-\data\images\normal"
    
    duplicates = find_duplicates(target_dir, threshold=2)
    
    if not duplicates:
        print("Không tìm thấy ảnh trùng lặp!")
        return

    print(f"\n📢 Tìm thấy {len(duplicates)} cặp ảnh tương đồng:")
    for dup, original, dist in duplicates:
        print(f"  - [{dist}] {dup.name} giống với {original.name}")

    confirm = input("\nBạn có muốn xóa các ảnh trùng lặp này không? (y/n): ")
    if confirm.lower() == 'y':
        for dup, original, dist in duplicates:
            try:
                os.remove(dup)
                print(f"Đã xóa: {dup.name}")
            except Exception as e:
                print(f" Không thể xóa {dup.name}: {e}")
        print("\n Đã dọn dẹp xong!")
    else:
        print("\n Đã hủy lệnh xóa.")

if __name__ == "__main__":
    main()


🔍 Đang quét 127 ảnh trong d:\Computer Vision\Computer-Vision Project\Computer-Vision-\data\images\normal...


  0%|          | 0/127 [00:00<?, ?it/s]

100%|██████████| 127/127 [00:00<00:00, 187.05it/s]

Không tìm thấy ảnh trùng lặp!
